In [1]:
import pandas as pd 
import os
import sys
import pandas as pd 
import numpy as np
sys.path.append('../src/')
sys.path.append('..')

In [2]:
from src.models.lib.data import *
from src.helper import *

In [3]:
files = os.listdir('../data/processed/labels/')
files

['allen_cortex_labels.csv',
 'primary_bhaduri_labels.csv',
 'allen_m1_region_labels.csv',
 'whole_brain_bhaduri_labels.csv']

In [4]:
def calculate_intersection(indata, genes, datapath):
     for datafile, labelfile in indata:
        dataset = GeneExpressionData(
            filename=datafile,
            labelname=labelfile,
        )

        currgenes = dataset.get_features()
        outfile = os.path.join(datapath, 'processed', f'{datafile[:-4]}_cleaned.csv')

        with open(outfile, mode='a+', encoding='utf-8') as f:
            for sample, _ in dataset: # Don't need label 
                temp = clean_sample(
                    sample=sample,
                    refgenes=genes,
                    currgenes=currgenes,
                ) 

                f.write('\n'.join(temp))



In [5]:
files = os.listdir('../data/processed/labels/')
test = GeneExpressionData(
    filename='../data/interim/allen_cortex_T.csv',
    labelname='../data/interim/labels/allen_cortex_labels.csv',
    class_label='Type',
    cast=True,
    skip=3
)

test[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 'Exclude')

In [6]:
refgenes = gene_intersection()

In [7]:
len(set(refgenes).intersection(test.get_features()))

1

In [8]:
from torch.utils.data import DataLoader, ConcatDataset

loader = DataLoader(test, batch_size=4)
sample = next(iter(loader))
sample = sample[0].numpy()

In [9]:
temp = np.array([[3,2,1],
        [1,2,3]])

np.sort(temp, axis=1)

array([[1, 2, 3],
       [1, 2, 3]])

In [10]:
def clean_sample(sample, refgenes, currgenes):
    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in currgenes that equal refgenes 
    
    sample = np.sort(sample, axis=1)
    sample = np.take(sample, indices, axis=1)

    return sample 

currgenes = test.get_features()

cleaned = clean_sample(
    sample=sample,
    refgenes=refgenes,
    currgenes=currgenes
)

In [11]:
cleaned

array([], shape=(4, 0), dtype=float32)

In [12]:
cleaned[0].sum()

0.0

In [13]:
# %%time

# for i, sample in enumerate(loader):
#     X, y = sample 
#     print(f'{i = }')
#     cleaned = clean_sample(
#         sample = X,
#         refgenes = refgenes,
#         currgenes = currgenes
#     )

In [14]:
len(test)

49494

In [15]:
datafiles, labelfiles = list(INTERIM_DATA_AND_LABEL_FILES_LIST.keys()), list(INTERIM_DATA_AND_LABEL_FILES_LIST.values())

datafiles = [os.path.join('..', 'data', 'interim', f) for f in datafiles]
labelfiles = [os.path.join('..', 'data', 'processed/labels', f) for f in labelfiles]
datafiles, labelfiles

(['../data/interim/primary_bhaduri_T.csv',
  '../data/interim/allen_cortex_T.csv',
  '../data/interim/allen_m1_region_T.csv',
  '../data/interim/whole_brain_bhaduri_T.csv'],
 ['../data/processed/labels/primary_bhaduri_labels.csv',
  '../data/processed/labels/allen_cortex_labels.csv',
  '../data/processed/labels/allen_m1_region_labels.csv',
  '../data/processed/labels/whole_brain_bhaduri_labels.csv'])

In [16]:
train, test, input_size, num_labels, weights = generate_datasets(
    datafiles,
    labelfiles,
    'Type'
)

In [17]:
trainloader = DataLoader(train, batch_size=4)

In [18]:
%%timeit 

sample = next(iter(trainloader))

18.1 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
# result = clean_sample(sample, refgenes, currgenes)

In [21]:
sample.shape

(4, 50281)

In [ ]:
clean_sample(sample, refgenes, currgenes)